Script Overview
Imports and Configurations:

Imports necessary libraries for various functionalities, such as data fetching, text-to-speech, and computer vision.
Sets up paths and configurations for models and Google Sheets.
Loading Models and Initial Setup:

Loads the pre-trained gender classification model and face detection model.
Initializes the webcam and the GPT-2 model for generating responses.
Function Definitions:

fetch_data_from_sheet(url): Fetches CSV data from Google Sheets.
preprocess_data(df): Preprocesses the DataFrame by handling empty rows and grouping product details.
generate_response(prompt, model, tokenizer, max_length=100): Generates a response from the GPT-2 model based on the given prompt.
extract_keywords_from_text(text, keywords): Extracts keywords from the provided text.
speak(text): Converts text to speech and plays it.
get_voice_input(): Captures voice input and converts it to text.
get_time_based_greeting(): Returns a time-based greeting (morning, afternoon, evening).
Main Function Logic:

Fetches data from Google Sheets and preprocesses it.
Initializes variables to manage face detection and greeting logic.
Continuously captures frames from the webcam, detects faces, and performs gender classification.
If a face is detected, it greets the detected person based on their gender and asks how it can help.
Captures voice input from the user, generates a response using the GPT-2 model, and provides relevant information based on keywords extracted from the response.
Displays the results and handles continuous updates until the user decides to exit.
End of Script:

Releases the webcam and closes the display window.

## Computer vision code with greeting

In [ ]:
import cv2
import os
import time
import pyttsx3
import datetime
import pytz

# Initialize text-to-speech engine
engine = pyttsx3.init()

# Define paths to the model files
prototxt_path = r'gender_pre_trained_models\gender_deploy.prototxt'
caffemodel_path = r'gender_pre_trained_models\gender_net.caffemodel'

# Check if files exist
if not os.path.isfile(prototxt_path):
    raise FileNotFoundError(f"Cannot find the prototxt file at {prototxt_path}")
if not os.path.isfile(caffemodel_path):
    raise FileNotFoundError(f"Cannot find the caffemodel file at {caffemodel_path}")

# Load the pre-trained Caffe models for gender classification
gender_net = cv2.dnn.readNetFromCaffe(prototxt_path, caffemodel_path)

# Define the mean values for the gender model
MODEL_MEAN_VALUES = (78.4263377603, 87.7689143744, 114.895847746)

# Gender list
gender_list = ['Male', 'Female']

# Load the pre-trained face detection model
face_cascade = cv2.CascadeClassifier(cv2.data.haarcascades + 'haarcascade_frontalface_default.xml')

# Open the webcam
cap = cv2.VideoCapture(0)

# Initialize variables to keep track of detection and greeting logic
last_detect_time = 0
greeting_interval = 2  # Interval to check if person is not available
greeted = False

def get_time_based_greeting():
    # Define the timezone for Sri Lanka
    sri_lanka_tz = pytz.timezone('Asia/Colombo')
    
    # Get the current time in Sri Lanka
    now = datetime.datetime.now(sri_lanka_tz)
    
    # Determine the greeting based on the current time
    hour = now.hour
    if 5 <= hour < 12:
        return "Good morning"
    elif 12 <= hour < 18:
        return "Good afternoon"
    else:
        return "Good evening"

while True:
    # Capture frame-by-frame
    ret, frame = cap.read()
    if not ret:
        break

    # Convert frame to grayscale
    gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)

    # Detect faces
    faces = face_cascade.detectMultiScale(gray, scaleFactor=1.1, minNeighbors=5, minSize=(30, 30))

    if len(faces) > 0:
        last_detect_time = time.time()

        if not greeted:
            for (x, y, w, h) in faces:
                # Extract the face region
                face_img = frame[y:y+h, x:x+w].copy()

                # Prepare the face image for classification
                blob = cv2.dnn.blobFromImage(face_img, 1.0, (227, 227), MODEL_MEAN_VALUES, swapRB=False)
                gender_net.setInput(blob)

                # Perform gender classification
                gender_preds = gender_net.forward()
                gender = gender_list[gender_preds[0].argmax()]

                # Draw a rectangle around the face and display the gender
                label = f'{gender}'
                cv2.rectangle(frame, (x, y), (x+w, y+h), (255, 0, 0), 2)
                cv2.putText(frame, label, (x, y-10), cv2.FONT_HERSHEY_SIMPLEX, 0.9, (255, 0, 0), 2)

                # Get the time-based greeting
                time_based_greeting = get_time_based_greeting()
                
                # Say the greeting based on detected gender
                if gender == 'Male':
                    engine.say(f"{time_based_greeting} Sir")
                elif gender == 'Female':
                    engine.say(f"{time_based_greeting} Miss")
                engine.runAndWait()
                
                # Set greeted to True to prevent repeated greetings
                greeted = True
                break  # Exit the loop after greeting
    else:
        # Check if the person has been missing for more than the greeting interval
        if time.time() - last_detect_time >= greeting_interval:
            greeted = False

    # Display the resulting frame
    cv2.imshow('Gender Classification', frame)

    # Break the loop if 'q' is pressed
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

# Release the webcam and close all OpenCV windows
cap.release()
cv2.destroyAllWindows()


## Chat bot english as seller

In [ ]:
import requests
import pandas as pd
from io import StringIO
from transformers import GPT2LMHeadModel, GPT2Tokenizer
import torch
import speech_recognition as sr
from gtts import gTTS
from playsound import playsound
import os

# Google Sheets configuration
SPREADSHEET_ID = '1lNZDwXKNlNkZiLWKksob55RE70e3IbSAhkR0W_pjGV4'
SHEET_NAME = 'Sheet1'  # Adjust if necessary
URL = f'https://docs.google.com/spreadsheets/d/{SPREADSHEET_ID}/gviz/tq?tqx=out:csv&sheet={SHEET_NAME}'

def fetch_data_from_sheet(url):
    """Fetches CSV data from the Google Sheet URL."""
    response = requests.get(url)
    if response.status_code == 200:
        return response.content.decode('utf-8')
    else:
        print('Failed to fetch data.')
        return None

def preprocess_data(df):
    """Preprocess the DataFrame to handle empty rows and group product details."""
    df = df.fillna('')
    df['Product'] = df['Product'].replace('', pd.NA).ffill()
    return df.dropna(subset=['Product'])

def generate_response(prompt, model, tokenizer, max_length=100):
    """Generates a response from the model based on the prompt."""
    inputs = tokenizer(prompt, return_tensors="pt", padding=True, truncation=True, max_length=max_length)
    input_ids = inputs["input_ids"]
    attention_mask = inputs["attention_mask"]

    with torch.no_grad():
        output_ids = model.generate(
            input_ids,
            attention_mask=attention_mask,
            max_length=max_length,
            num_return_sequences=1,
            do_sample=True,
            temperature=0.7,
            top_p=0.9,
            pad_token_id=tokenizer.pad_token_id,
        )

    generated_text = tokenizer.decode(output_ids[0], skip_special_tokens=True)
    return generated_text

def extract_keywords_from_text(text, keywords):
    """Extracts multi-word keywords from the given text."""
    text_lower = text.lower()
    extracted_keywords = []
    
    for keyword in keywords:
        keyword_lower = keyword.strip().lower()
        if keyword_lower in text_lower:
            extracted_keywords.append(keyword_lower)
    
    return extracted_keywords

def speak(text):
    """Convert text to speech and play it."""
    tts = gTTS(text=text, lang='en')
    tts.save("response.mp3")
    playsound("response.mp3")
    os.remove("response.mp3")

def get_voice_input():
    """Capture voice input and convert it to text."""
    recognizer = sr.Recognizer()
    with sr.Microphone() as source:
        print("Listening...")
        audio = recognizer.listen(source)
    try:
        text = recognizer.recognize_google(audio)
        print(f"You: {text}")
        return text
    except sr.UnknownValueError:
        print("Sorry, I did not understand that.")
        speak("Sorry, I did not understand that.")
        return ""
    except sr.RequestError:
        print("Sorry, my speech service is down.")
        speak("Sorry, my speech service is down.")
        return ""

def main():
    # Load the trained model and tokenizer
    model_name = 'custom_seller_bot_model'  # Directory where your trained model is saved
    tokenizer = GPT2Tokenizer.from_pretrained(model_name)
    model = GPT2LMHeadModel.from_pretrained(model_name)
    model.eval()

    # Fetch data from Google Sheets
    csv_data = fetch_data_from_sheet(URL)
    if not csv_data:
        print("Failed to fetch data from the Google Sheet.")
        return
    
    # Convert CSV data to DataFrame
    df = pd.read_csv(StringIO(csv_data))
    
    # Preprocess the data
    df = preprocess_data(df)
    
    # Extract keywords from the DataFrame
    keywords = df['Product'].dropna().unique().tolist()
    keywords = [keyword.strip() for keyword in keywords]  # Remove extra spaces

    # Debug: Print keywords and DataFrame contents
    print(f"Keywords from DataFrame: {keywords}")
    print(f"DataFrame contents:\n{df}")

    while True:
        # Get user input
        user_input = get_voice_input()
        if user_input.lower() == 'exit':
            print("Exiting...")
            speak("Exiting.")
            break

        if user_input.strip() == "":
            continue

        # Generate response from the model
        prompt = user_input
        model_response = generate_response(prompt, model, tokenizer)
        print(f"Model response: {model_response}")

        # Split model response into sentences
        sentences = model_response.split('.')
        
        if len(sentences) > 1:
            # Extract the second sentence
            second_sentence = sentences[1].strip()
        else:
            # Handle case where there is no second sentence
            second_sentence = sentences[0].strip()
        
        # Extract keywords from the entire model response
        matched_keywords = extract_keywords_from_text(model_response, keywords)
        
        # Speak the second sentence of the model response if it exists
        if second_sentence:
            speak(second_sentence)

        # Handle keywords and provide pricing information
        if matched_keywords:
            for keyword in matched_keywords:
                # Check for matches in DataFrame
                matched_rows = df[df['Product'].str.contains(keyword, case=False, na=False)]
                if not matched_rows.empty:
                    for _, row in matched_rows.iterrows():
                        product_name = row['Product']
                        brand_type = row['BrandType']
                        price = row['Price']
                        details = f"We have {product_name}, {brand_type} with the price {price}"
                        print(details)
                        speak(details)
                else:
                    print(f"Sorry, we don't have information on {keyword}.")
                    speak(f"Sorry, we don't have information on {keyword}.")
        else:
            if second_sentence:
                print("Sorry, I couldn't find any matching items.")
                speak("Sorry, I couldn't find any matching items.")

        speak("Is there any other information you want to know?")

if __name__ == '__main__':
    main()


## Model training

In [ ]:
import pandas as pd
from datasets import Dataset
from transformers import Trainer, TrainingArguments, GPT2LMHeadModel, GPT2Tokenizer

# Use the correct file path
file_path = 'prepared_customer_seller_data.csv'

try:
    # Load the CSV file into a pandas DataFrame
    df = pd.read_csv(file_path)
    
    # Combine input and response text for causal language modeling
    df['text'] = df['input_text'] + " " + df['response_text']
    
    # Convert the DataFrame to a Dataset
    dataset = Dataset.from_pandas(df[['text']])
    
    # Initialize the tokenizer and model
    model_name = 'gpt2'  # or 'gpt2-small', 'gpt2-medium', 'gpt2-large', etc.
    tokenizer = GPT2Tokenizer.from_pretrained(model_name)
    model = GPT2LMHeadModel.from_pretrained(model_name)
    
    # Add a padding token if not already present
    if tokenizer.pad_token is None:
        tokenizer.pad_token = tokenizer.eos_token  # Use the end-of-sequence token as padding token

    # Tokenize the combined text
    def tokenize_function(example):
        encodings = tokenizer(example["text"], padding="max_length", truncation=True, max_length=512)
        encodings["labels"] = encodings["input_ids"].copy()  # Use input_ids as labels
        return encodings
    
    tokenized_datasets = dataset.map(tokenize_function, batched=True, remove_columns=["text"])
    
    # Prepare training arguments
    training_args = TrainingArguments(
        output_dir='./results',
        per_device_train_batch_size=1,
        num_train_epochs=3,
        logging_dir='./logs',
        logging_steps=10,
        save_steps=500,
        save_total_limit=2,
        report_to=None,  # Ensure you're not using any unsupported reporting options
    )
    
    # Initialize the Trainer
    trainer = Trainer(
        model=model,
        args=training_args,
        train_dataset=tokenized_datasets,
    )
    
    # Train the model
    trainer.train()
    
    # Save the model
    model.save_pretrained('custom_seller_bot_model')
    tokenizer.save_pretrained('custom_seller_bot_model')
    
    print("Training complete and model saved.")
    
except FileNotFoundError:
    print(f"The file at {file_path} was not found. Please check the path and try again.")
except Exception as e:
    print(f"An error occurred: {e}")
